# 1.데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

In [21]:
train = pd.read_csv('/aiffel/train.csv')
train.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [22]:
daily = pd.read_csv('/aiffel/daily.csv')
daily.drop(['Unnamed: 0'], axis=1, inplace=True)
daily.head()

,idx,class,conversation
0,3950,일반 대화,형 고양이 키워본 적 있나?\n엉 키키 길고양이 데리고 와서 키워봤다\n키키 길고양...
1,3951,일반 대화,안녕하세요 #@소속# #@이름#입니다\n수고하십니다 문의할 게 있어요\n네 반갑습니...
2,3952,일반 대화,오빠야는 몇 살 때 승용차 샀어 ?\n23살 때 중고로 승용차 구매 했징\n오 23...
3,3953,일반 대화,다음 주면 결혼랑 2주년인데 선물 뭐 하지\n벌써 2년이야?\n가방 사드려\n그게 ...
4,3954,일반 대화,반갑습니다 #@소속# 상담원 #@이름#입니다\n 안녕하세요 중학교 강의 문의 좀 ...


In [23]:
daily.reset_index(inplace=True, drop=True)
train.reset_index(inplace=True, drop=True)
train = pd.concat([train, daily])
train.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [25]:
train.shape

(5000, 3)

In [31]:
data = train

In [32]:
!pip install transformers

In [33]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import re
import tensorflow as tf

# 2.라벨인코딩

In [35]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(data['class'])
label_encoder.classes_

array(['갈취 대화', '기타 괴롭힘 대화', '일반 대화', '직장 내 괴롭힘 대화', '협박 대화'],
      dtype=object)

In [36]:
data['class'] = label_encoder.transform(data['class'])
data.head()

,idx,class,conversation
0,0,4,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,4,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,1,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,0,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,0,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


# 3.train, validation 분리

In [41]:
from sklearn.model_selection import train_test_split

train = data['conversation'].tolist()
target = data['class'].tolist()

# train, val 분리 (20%)
train_x, val_x, train_y, val_y = \
train_test_split(train, target, random_state=27, test_size=0.2)

# val, test 분리 (test를 전체의 10%)
val_x, test_x, val_y, test_y = train_test_split(val_x, val_y, random_state=27, test_size=0.5)

len(train_x), len(val_x), len(test_x)

(4000, 500, 500)

# 4.토크나이징

In [42]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

train_tensor = tokenizer(train_x, truncation=True, padding=True)
val_tensor = tokenizer(val_x, truncation=True, padding=True)

# 5.입력데이터셋 생성

In [43]:
# trainset-set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_tensor),
    train_y
))

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_tensor),
    val_y
))

# 6.모델 컴파일

In [45]:
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=5, from_pt=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  110617344 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3845      
Total params: 110,621,189
Trainable params: 110,621,189
Non-trainable params: 0
_________________________________________________________________


# 7.훈련

In [52]:
history = model.fit(
    train_dataset.shuffle(len(train_dataset)).batch(8),
    validation_data = val_dataset.shuffle(len(val_dataset)).batch(8),
    epochs=1, 
    batch_size=8
)

500/500 [==============================] - 511s 1s/step - loss: 0.3681 - accuracy: 0.8760 - val_loss: 0.2304 - val_accuracy: 0.9380


In [53]:
id2labels = model.config.id2label
model.config.id2label = {id : label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0]  for id, label in id2labels.items()}

label2ids = model.config.label2id
model.config.label2id = {label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] : id   for id, label in id2labels.items()}

model.config.id2label

{0: '갈취 대화', 1: '기타 괴롭힘 대화', 2: '일반 대화', 3: '직장 내 괴롭힘 대화', 4: '협박 대화'}

# 8.모델저장

In [54]:
MODEL_SAVE_PATH = '/aiffel/klue-bert data'

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

('/aiffel/klue-bert data/tokenizer_config.json',
 '/aiffel/klue-bert data/special_tokens_map.json',
 '/aiffel/klue-bert data/vocab.txt',
 '/aiffel/klue-bert data/added_tokens.json')

# 9. 모델  로드

In [87]:
from transformers import TextClassificationPipeline

# Load Fine-tuning model
loaded_tokenizer = BertTokenizer.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='tf',
)

Some layers from the model checkpoint at /aiffel/klue-bert data were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /aiffel/klue-bert data.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


# 10.예측

In [90]:
predicted_list = []

for text in test_x:
    # predict
    preds_list = text_classifier(text)[0]
    sorted_preds_list = preds_list['label']
    predicted_list.append(sorted_preds_list)

In [108]:
test_df = pd.DataFrame(predicted_list)
test_df['true_label'] = test_y
test_df = test_df[['true_label',0]]
test_df.head()

,true_label,0
0,협박 대화,갈취 대화
1,일반 대화,일반 대화
2,직장 내 괴롭힘 대화,직장 내 괴롭힘 대화
3,직장 내 괴롭힘 대화,직장 내 괴롭힘 대화
4,기타 괴롭힘 대화,기타 괴롭힘 대화


In [109]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_true=test_df['true_label'], y_pred=test_df[0]))

              precision    recall  f1-score   support

       갈취 대화       0.87      0.88      0.88        78
   기타 괴롭힘 대화       0.93      0.82      0.87       106
       일반 대화       1.00      1.00      1.00       123
 직장 내 괴롭힘 대화       0.95      0.97      0.96       107
       협박 대화       0.81      0.88      0.84        86

    accuracy                           0.92       500
   macro avg       0.91      0.91      0.91       500
weighted avg       0.92      0.92      0.92       500

